<a href="https://colab.research.google.com/github/kiranbeethoju/Stock-Market-Forecast/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

start = '2004-02-01'
end = '2020-07-04'

tickers = ['BHARTIARTL.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(names)


                   ticker        Open   Adj Close
Date                                             
2004-02-03  BHARTIARTL.NS   62.298401   55.406151
2004-02-04  BHARTIARTL.NS   58.993000   56.486176
2004-02-05  BHARTIARTL.NS   60.576801   55.449345
2004-02-06  BHARTIARTL.NS   59.865200   60.395962
2004-02-09  BHARTIARTL.NS   64.731598   60.007133
...                   ...         ...         ...
2020-06-29  BHARTIARTL.NS  558.900024  567.099976
2020-06-30  BHARTIARTL.NS  572.000000  559.849976
2020-07-01  BHARTIARTL.NS  575.000000  560.150024
2020-07-02  BHARTIARTL.NS  564.799988  558.450012
2020-07-03  BHARTIARTL.NS  561.799988  581.450012

[4044 rows x 3 columns]


In [52]:
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]

In [53]:
close.describe()

,y
count,4044.000000
mean,290.193860
std,98.080078
min,54.498901
25%,263.803673
50%,301.631317
75%,346.412720
max,598.799988


In [54]:
!pip install fbprophet

In [55]:
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df):
  df = df[4000:]
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=430)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  return mixed

In [56]:
a

,ds,actual_open,yhat_future_open,yhat_lower_future_open,yhat_upper_future_open,pred_open_avg
0,2020-05-04,502.450012,523.990473,506.608315,541.462137,524.02
1,2020-05-05,538.000000,531.392060,513.962267,548.645018,531.333
2,2020-05-06,531.750000,539.196597,521.669874,556.921813,539.263
3,2020-05-07,545.349976,535.702291,518.594922,552.786986,535.695
4,2020-05-08,540.000000,536.371158,518.688551,553.473884,536.178
5,2020-05-11,533.049988,540.643502,523.265764,558.003565,540.638
6,2020-05-12,533.000000,548.045090,529.448602,565.963095,547.819
7,2020-05-13,568.400024,555.849627,538.837684,574.001505,556.23
8,2020-05-14,551.099976,552.354709,535.184744,569.450224,552.33
9,2020-05-15,543.500000,553.022963,535.175344,571.121506,553.107


In [57]:
a = run(open)
b = run(close)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [58]:
a = a.rename(columns={"avg_val":"pred_open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_open","yhat_future":"yhat_future_open","yhat_lower_future":"yhat_lower_future_open"})
b = b.rename(columns={"avg_val":"pred_close_avg","y":"actual_close","yhat_upper_future":"yhat_upper_future_close","yhat_future":"yhat_future_close","yhat_lower_future":"yhat_lower_future_close"})


In [59]:
a.columns,b.columns

(Index(['ds', 'actual_open', 'yhat_future_open', 'yhat_lower_future_open',
        'yhat_upper_future_open', 'pred_open_avg'],
       dtype='object'),
 Index(['ds', 'actual_close', 'yhat_future_close', 'yhat_lower_future_close',
        'yhat_upper_future_close', 'pred_close_avg'],
       dtype='object'))

In [60]:
c = pd.merge(a,b,on="ds",how="inner")

In [65]:
c.tail()

,ds,actual_open,yhat_future_open,yhat_lower_future_open,yhat_upper_future_open,pred_open_avg,actual_close,yhat_future_close,yhat_lower_future_close,yhat_upper_future_close,pred_close_avg
39,2020-06-29,558.900024,559.889990,544.341779,577.015521,560.416,567.099976,561.194672,545.055619,577.395529,561.215
40,2020-06-30,572.000000,565.783822,550.249039,581.228069,565.754,559.849976,564.918925,547.930411,582.012629,564.954
41,2020-07-01,575.000000,570.795764,553.948337,587.798211,570.847,560.150024,564.514433,547.201094,582.020431,564.579
42,2020-07-02,564.799988,564.298429,547.015256,580.675092,563.996,558.450012,560.554778,544.198789,577.656154,560.803
43,2020-07-03,561.799988,562.300624,545.074333,578.591664,561.989,581.450012,566.796157,550.482107,582.559141,566.612


In [62]:
df = c[["ds","actual_open","pred_open_avg","actual_close","pred_close_avg"]]

In [63]:
df.tail()

,ds,actual_open,pred_open_avg,actual_close,pred_close_avg
39,2020-06-29,558.900024,560.416,567.099976,561.215
40,2020-06-30,572.000000,565.754,559.849976,564.954
41,2020-07-01,575.000000,570.847,560.150024,564.579
42,2020-07-02,564.799988,563.996,558.450012,560.803
43,2020-07-03,561.799988,561.989,581.450012,566.612


In [69]:
df["open_diff"] = df["pred_open_avg"]-df["actual_open"]
df["close_diff"] = df["pred_close_avg"]-df["actual_close"]
df["actual_raise_of_value"] = df["actual_close"] - df["actual_open"]
df["predicted_raise_of_value"] = df["pred_close_avg"] - df["pred_open_avg"]
df.tail(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,ds,actual_open,pred_open_avg,actual_close,pred_close_avg,open_diff,close_diff,probable_raise_of_value,actual_raise_of_value,predicted_raise_of_value
34,2020-06-22,575.799988,560.384,581.049988,561.669,-15.416,-19.3808,5.250000,5.250000,1.28512
35,2020-06-23,581.049988,566.394,578.799988,565.822,-14.656,-12.9781,-2.250000,-2.250000,-0.572137
36,2020-06-24,581.000000,571.551,557.349976,564.607,-9.44929,7.25666,-23.650024,-23.650024,-6.94407
37,2020-06-25,553.900024,565.699,559.000000,560.929,11.7988,1.92926,5.099976,5.099976,-4.76959
38,2020-06-26,560.000000,563.371,560.400024,567.088,3.37076,6.68811,0.400024,0.400024,3.71737
39,2020-06-29,558.900024,560.416,567.099976,561.215,1.51574,-5.8847,8.199951,8.199951,0.79951
40,2020-06-30,572.000000,565.754,559.849976,564.954,-6.24636,5.10401,-12.150024,-12.150024,-0.799655
41,2020-07-01,575.000000,570.847,560.150024,564.579,-4.15256,4.42863,-14.849976,-14.849976,-6.26878
42,2020-07-02,564.799988,563.996,558.450012,560.803,-0.803729,2.35323,-6.349976,-6.349976,-3.19302
43,2020-07-03,561.799988,561.989,581.450012,566.612,0.188886,-14.8375,19.650024,19.650024,4.62359
